In [ ]:
from core import initialize_bodies
from core import calculate_force
from core import Body
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import colormaps

In [ ]:
# initialize bodies with random masses, positions, and velocities over set range
# bodies[n] = Body(mass, position [x, y], velocity [vx, vy])
bodies = initialize_bodies(30, mass_range = [0.1, 1], position_range = [-32, 32], velocity_range = [-0.01, 0.01])

# using dimensionless units in terms of solar masses, AU

# solar system
'''bodies = [
    Body(mass = 1, position=[0, 0], velocity=[0, 0]), # sun
    Body(mass = 3.00e-6, position=[1, 0], velocity=[0, 1]), # earth
    Body(mass = 3.21e-7, position=[1.5, 0], velocity=[0, np.sqrt(1/1.5)]), # mars
    Body(mass = 9.55e-4, position=[5.2, 0], velocity=[0, np.sqrt(1/5.2)]), # jupiter
    Body(mass = 2.86e-4, position=[9.5, 0], velocity=[0, np.sqrt(1/9.5)]), # saturn
    Body(mass = 4.36e-5, position=[19.2, 0], velocity=[0, np.sqrt(1/19.2)]), # uranus
    Body(mass = 5.15e-5, position=[30.1, 0], velocity=[0, np.sqrt(1/30.1)]) # neptune
]'''

# binary star system
'''bodies = [
    Body(mass = 2, position = [0, 10], velocity = [-0.1, 0]), # star_1
    Body(mass = 1, position = [0, -10], velocity = [0.1, 0]) # star_2
]'''

In [ ]:
# Animation using matplotlib

%matplotlib widget

# Time step for position updates
dt = 0.1

# Set up the plot
#fig, ax = plt.subplots()
fig, ax = plt.subplots(figsize=(19.2, 10.8), dpi=100)
ax.set_xlim(-32, 32) # X-axis limits
ax.set_ylim(-32, 32) # Y-axis limits
ax.set_title("Particle Simulation", color="black")  # Title
ax.set_xlabel("X Position", color="black")  # X-axis label
ax.set_ylabel("Y Position", color="black")  # Y-axis label
ax.tick_params(axis='x', colors="black")  # X-axis ticks
ax.tick_params(axis='y', colors="black")  # Y-axis ticks
ax.set_facecolor("black")  # Set background color
fig.patch.set_facecolor("lightgray")  # Outside the plot
ax.grid(color="gray", linestyle="--", linewidth=0.5, alpha=0.5)

#cmap = colormaps["tab20"].resampled(len(bodies))  # Dynamically resample colormap
#colors = [cmap(i) for i in range(len(bodies))] # Create list of colors
colors = ['yellow', 'blue', 'red', 'orange', 'goldenrod', 'lightblue', 'mediumblue'] # planet colors

# Create scatter plot with initial positions
scat = ax.scatter(
    [body.position[0] for body in bodies], # Initial x-coordinate of bodies
    [body.position[1] for body in bodies], # Initial y-coordinate of bodies
    s = [50*(body.mass)**(1/3) for body in bodies], # Size of bodies proportional to mass
    c = 'yellow',  # Color of bodies
)

# Update function for animation
def update_position(frame):

    for n, body in enumerate(bodies):

        F = calculate_force(bodies) # calculate net force on bodies

        # Leapfrog method
        body.velocity += 0.5 * F[n] / body.mass * dt # Update velocity (half step)
        body.position += body.velocity * dt # Update position (full step)
        F = calculate_force(bodies) # Recalculate force
        body.velocity += 0.5 * F[n] / body.mass * dt # Update velocity (second half step)
        positions = [body.position for body in bodies] # Create array of positions to update scatter plot
    scat.set_offsets(positions)  # Update scatter plot
  
    return scat

# Create the animation
ani = animation.FuncAnimation(
    fig=fig,
    func=update_position,
    frames=range(1000),  # Number of frames
    interval=1,  # Time between frames in milliseconds
    repeat=False,  # Stop after the last frame
)

ani.save('direct_simulation.mp4', writer='ffmpeg', fps=30) # Save animation as .mp4

# Show the plot
plt.show()

In [ ]:
F = calculate_force(bodies)

for i, body in enumerate(bodies):
    print(f'Net foce on body_{i} Fx = {F[i,0]} Fy = {F[i,1]})')

In [ ]:
print(bodies[0].position[0])

In [ ]:
sim_data =[(34.0,5), (36.6,10), (46.9,15), (66.1,20), (99.1,25), (146.4,30)]